In [2]:
#Loading libraries
import pandas as pd 

import os
import re
import nltk
from sklearn import feature_extraction
from tqdm import tqdm


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# NLTK
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline
import seaborn as sns
from nltk.stem import WordNetLemmatizer

from gensim.models.word2vec import Word2Vec
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit

/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
df_all = pd.read_csv('./preprocessed.csv')

In [19]:
df_all.head(5)

,Unnamed: 0,Unnamed: 0.1,Headline,Body ID,Stance,articleBody,stance_dummy
0,0,0,police find mass graf least body near mexico ...,712,unrelated,danny boyle directing untitled film seth rogen...,3
1,1,1,seth rogen play apple steve wozniak,712,discuss,danny boyle directing untitled film seth rogen...,2
2,2,2,mexico police find mass grave near site stude...,712,unrelated,danny boyle directing untitled film seth rogen...,3
3,3,3,mexico say missing student found first mass graf,712,unrelated,danny boyle directing untitled film seth rogen...,3
4,4,4,new io bug delete icloud document,712,unrelated,danny boyle directing untitled film seth rogen...,3


In [55]:
stance_dummy = pd.get_dummies(df_all['Stance'], drop_first=False)
stance_dummy.head()

,agree,disagree,discuss,unrelated
0,0,0,0,1
1,0,0,1,0
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


## GloVe

Using word embeddings on both the headline and article bodies. This pretrained model is called the Global Vectors for Word Representation or GloVE. It is pretrained on 6 billion words and can use its 400,000 word vocabulary for vectors of 50, 100, 200, or 300.
I use a keras preprocessor to turn words in to integer sequence which creates an index of all the words, selects the top most used words to track, then turns corpus into list of word sequences represented by integer index.

In [56]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

max_sequence_length = 100   # max number of words to consider in a review (i.e. the first 50...)
max_num_words = 4000       # maximum number of words to include in the vocabulary (i.e. top 10000 only)

# Instantiate the Tokenizer
tokenizer = Tokenizer(num_words=max_num_words)

# Fit the tokenizer, i.e. learn the vocab and id the most frequently occuring words
tokenizer.fit_on_texts(df_all['Headline'].values)

# Turn our texts to sequences of word indices
sequences = tokenizer.texts_to_sequences(df_all['Headline'].values)

# Save the look-up dictionary for words to indices (will need this later)
word_index = tokenizer.word_index

# Pad out our sequences by prepending zeros to all of our text sequences
padded_sequences_headline = pad_sequences(sequences, maxlen=max_sequence_length)

print(f'Found {len(word_index)} unique tokens/words')
print(f'The maximum word index is {padded_sequences_headline.max()}')

Found 2903 unique tokens/words
The maximum word index is 2903


In [57]:
padded_sequences_headline.shape

(49972, 100)

In [58]:
padded_sequences_headline[0,:]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  208,
        251,  102,  311,  138,  245,  165,   80,  878,   59, 1942,  208,
       1943], dtype=int32)

In [59]:
max_sequence_length = 200   # max number of words to consider in a review (i.e. the first 50...)
max_num_words = 25000     # maximum number of words to include in the vocabulary (i.e. top 10000 only)

# Instantiate the Tokenizer
tokenizer = Tokenizer(num_words=max_num_words)

# Fit the tokenizer, i.e. learn the vocab and id the most frequently occuring words
tokenizer.fit_on_texts(df_all['articleBody'].values)

# Turn our texts to sequences of word indices
sequences = tokenizer.texts_to_sequences(df_all['articleBody'].values)

# Save the look-up dictionary for words to indices (will need this later)
word_index = tokenizer.word_index

# Pad out our sequences by prepending zeros to all of our text sequences
padded_sequences_articleBody = pad_sequences(sequences, maxlen=max_sequence_length)

print(f'Found {len(word_index)} unique tokens/words')
print(f'The maximum word index is {padded_sequences_articleBody.max()}')

Found 19876 unique tokens/words
The maximum word index is 19876


In [60]:
padded_sequences_articleBody.shape

(49972, 200)

In [61]:
padded_sequences_articleBody

array([[   0,    0,    0, ...,  278,    8, 1073],
       [   0,    0,    0, ...,  278,    8, 1073],
       [   0,    0,    0, ...,  278,    8, 1073],
       ...,
       [   0,    0,    0, ...,  159,  245, 2899],
       [3013, 1672,  518, ...,  157,  187,  960],
       [   0,    0,    0, ...,  292,  266,  733]], dtype=int32)

Combining the padded sequences of both headline and article body together before running model. 

In [62]:
padded_sequences = np.concatenate((padded_sequences_headline, padded_sequences_articleBody), axis=1)
padded_sequences.shape

(49972, 300)

In [63]:
df_all['stance_dummy'].shape

(49972,)

In [65]:
# Train, test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, stance_dummy, test_size=0.33, random_state=42)

X_train.shape, y_train.shape

((33481, 300), (33481, 4))

In [66]:
#reading in GloVe
import numpy as np

glove_dir = '../glove.6B (1)/'

embeddings_index = {}

with open(os.path.join(glove_dir, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
print(f'Found {len(embeddings_index)} word vectors')

Found 400000 word vectors


Create matrix for word vectors for words in tokenized vocabulary. 

In [67]:
embedding_dim = 100

embedding_matrix = np.zeros((max_num_words, embedding_dim))
for word, index in word_index.items():
    if index < max_num_words:
        # Using "get" with the dict is a safe way to avoid missing key errors
        embedding_vector = embeddings_index.get(word) 
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [68]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(max_num_words, embedding_dim, input_length=300))
model.add(LSTM(20))
model.add(Dense(4, activation='softmax'))

Use pretrained GloVe weights in matrix(embedding matrix).

In [69]:
model.layers[0].set_weights([embedding_matrix])

Embedding layer is input for LSTM. 

In [70]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 100)          2500000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 20)                9680      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 84        
Total params: 2,509,764
Trainable params: 2,509,764
Non-trainable params: 0
_________________________________________________________________


Train neural net

In [71]:
history = model.fit(X_train, y_train, epochs=10, batch_size=300, validation_data=(X_test, y_test))

Train on 33481 samples, validate on 16491 samples
Epoch 1/10
33481/33481 [==============================] - 100s 3ms/step - loss: 0.3712 - acc: 0.8481 - val_loss: 0.3268 - val_acc: 0.8662
Epoch 2/10
33481/33481 [==============================] - 90s 3ms/step - loss: 0.2931 - acc: 0.8840 - val_loss: 0.2863 - val_acc: 0.8891
Epoch 3/10
33481/33481 [==============================] - 81s 2ms/step - loss: 0.2618 - acc: 0.8991 - val_loss: 0.2712 - val_acc: 0.8950
Epoch 4/10
33481/33481 [==============================] - 80s 2ms/step - loss: 0.2474 - acc: 0.9045 - val_loss: 0.2628 - val_acc: 0.9003
Epoch 5/10
33481/33481 [==============================] - 81s 2ms/step - loss: 0.2373 - acc: 0.9074 - val_loss: 0.2568 - val_acc: 0.9027
Epoch 6/10
33481/33481 [==============================] - 89s 3ms/step - loss: 0.2306 - acc: 0.9096 - val_loss: 0.2522 - val_acc: 0.9026
Epoch 7/10
33481/33481 [==============================] - 90s 3ms/step - loss: 0.2256 - acc: 0.9106 - val_loss: 0.2500 - val_ac

In [72]:
# What's baseline?
np.unique(y_train, return_counts=True)[1]/len(y_train)

array([3., 1.])

For prediction, passing neural net a sequence of word indices with sequence length equal to max sequence length. 

In [73]:
X_test[0:1,:].shape

(1, 300)

In [74]:
model.predict(X_test[0:1,:])

array([[0.01394238, 0.0048189 , 0.00776374, 0.973475  ]], dtype=float32)

# Executive Summary 

My goal was to classify my data(headlines and articles) by stance detection. Stance detection classifies the stance of the body text relative to the claim made in the headline into four categories: agree, disagree, discuss, and unrelated. Stance detection is currently used to verify what other news organizations are saying about a topic and is the first step in automating detection of fake news. The version of stance detection I will be using is building on the work of William Ferreira and Andreas Vlachos. I recieved my data from the data provided by the FakeNewsChallenge.org. The data is taken from the Emergent Dataset created by Craig Silverman, an expert in "Fake News". Emergent is a "real time rumor tracker" focusing on how rumors are reported in the media. 

My model begins with the baseline class at 73%. Through implementation of count vectorizer, TF-IDF, and using Logistic Regression and Random Trees to classify my metrics moved little past 73%. While it seemed that no form of vectorization was helping increase the score, the ROC model showed the micro average at 88% giving me hope that a higher score could be reached in some way. That is when I learned about word2vec and using LSTMs for a neural network. After some failures with word2vec, I implemented the GloVe model which is far superior to word2vec because it has been pretrained on many more words than just my dataset. Using word embeddings, I was able to train a neural network that scored 90% accuracy. 

My findings were that word embeddings and neural networks were necessary to create a model above baseline to predict stance. Dummying the stance label also helped tighten the results. If given more time, I would have run more classification models and tried to train my own neural network with more data. I would have also liked to have predicted more specifically for the desired classes. While my score currently exceeds what is on Kaggle, it is because I was not predicting for the agree and disagree and rather all the stances. I would have also have liked to play with LDA similarities some more to see what could be done with those. 
